In [1]:
#find snps
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import allel
import bokeh
import iqplot
import numpy as np
import scipy.stats as st
#import /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/permutations/results.csv
#read in data
snps = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/GWAS/snp_hunting.csv')
#washing phenotype /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/washing_phenotypes.csv
wash_corr = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Analysis of Clement-Ziza RNA-seq/wash_corr.csv')
#full snp dataset /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/SupplementaryDataset_S7_genotype.tsv
orthologs = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Orthologs/final_orthology_database.csv')
#import /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Phenotypes/phenotype_gwas.csv
wash_data = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Phenotypes/phenotype_yes_gwas.csv')


snpsdata = '/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Jeffares/Jeffares SNPs/ERZ289255/Spombe.2013-01-02.filt3c.nr57-final.snps.anno-snpeff3.cleaned4.vcf'
indelsdata = '/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Jeffares/Jeffares SNPs/ERZ289254/Spombe.indels.unique.nr57.final.annotated.2014-01-23.cleaned4.vcf.gz'
snpsdata = allel.read_vcf(snpsdata,fields=['samples','variants/CHROM','variants/POS','calldata/GT'])
indelsdata = allel.read_vcf(indelsdata,fields=['samples','variants/CHROM','variants/POS','calldata/GT'])



In [2]:
#pandas read in "/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Pombe genome annotation.gff3"
import pandas as pd
df = pd.read_csv('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Pombe genome annotation.gff3', sep='\t', header=1)
#break last column into 2 at ;
df[['a','b']] = df["ID=SPAC1002.01;Name=mrx11"].str.split(';',expand=True)
#remove column ID=SPAC1002.01;Name=mrx11
df.drop(columns =["ID=SPAC1002.01;Name=mrx11"], inplace = True)
#remove ID= from column a and rename it to ID
df['ID'] = df['a'].str.replace('ID=', '')
#remove column a
df.drop(columns =['a','PomBase',".","..1"], inplace = True)
#rename b to Parent
df.rename(columns={'b':'Parent','gene':'object','I':'chromosome','1798347':"start",'1798835':"end","+":"strand"}, inplace=True)
#In parent remove anything before =
df['Parent'] = df['Parent'].str.replace('Parent=', '')
#or Name=
df['Parent'] = df['Parent'].str.replace('Name=', '')
#create an aempty column called detail
df['detail'] = ""
#split ID column at :, and if there is something after put it in detail column
df['detail'] = df['ID'].str.split(':', expand=True)[1]
#split ID column at :, and if there is something before put it in ID column
df['ID'] = df['ID'].str.split(':', expand=True)[0]

In [3]:
#find a subset of df, where snps[position] is +- 1000 of df[start] or df[end]. Create a result df, and say which snps were there
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

result = pd.DataFrame()
for i in range(len(snps)):
    for j in range(len(df)):
        if snps['position'][i] >= df['start'][j] - 1500 and snps['position'][i] <= df['end'][j] + 1500 :
            #and if chromsome is chromosome_2 for df chromsome II etc
            if snps['chromosome'][i] == df['chromosome'][j]:
                result = result.append({'position': snps['position'][i], 'ID': df['ID'][j], 'Parent': df['Parent'][j], 'object': df['object'][j], 'chromosome': df['chromosome'][j], 'start': df['start'][j], 'end': df['end'][j], 'strand': df['strand'][j], 'detail': df['detail'][j], "p_value": snps["p_value"][i], "type":snps["type"][i]}, ignore_index=True)
result

#if the last two characters are .1, remove them
for i in range(len(result)):
    if result['ID'][i][-2:] == ".1":
        result['ID'][i] = result['ID'][i][:-2]



In [4]:
#match ID with systematic name in wash_corr and add columns gene	correlation	p-value
#create np.arrays with NAs
corrs = np.array([np.nan]*len(result))
pvals = np.array([np.nan]*len(result))
genes = np.array([np.nan]*len(result), dtype=object)
for i in range(len(result)):
    for j in range(len(wash_corr)):
        if result['ID'][i] == wash_corr['systematic_names'][j]:
            corrs[i] = wash_corr['correlation'][j]
            pvals[i] = wash_corr['p-value'][j]
            genes[i] = wash_corr['gene'][j]

result['correlation'] = corrs
result['p_value_corr'] = pvals
result['gene'] = genes

In [5]:

result_final__ = result.copy()

In [6]:
#iterate through result_final__. If ID is found in orthologs, then add the albicans_name(s)cerevisiae_name(s) to the df
from numpy import dtype

albicans_names = np.array([np.nan]*len(result_final__),dtype=object)
cerevisiae_names = np.array([np.nan]*len(result_final__),dtype=object)
for i in range(len(result_final__)):
    for j in range(len(orthologs)):
        if result_final__['ID'][i] in orthologs['pombe_name(s)'][j]:
            albicans_names[i] = orthologs['albicans_name(s)'][j]
            cerevisiae_names[i] = orthologs['cerevisiae_name(s)'][j]
#add to df
result_final__['albicans_name(s)'] = albicans_names
result_final__['cerevisiae_name(s)'] = cerevisiae_names

In [7]:
#iterate through positions in result_final__  and find the corresponding alleles in snps. Then add the two values in X968_allele	Y0036_allele to the df in two new columns at the start 
#add two columns to result_final__
result_final__['allele_1'] = np.nan
result_final__['allele_2'] = np.nan
for i in range(len(result_final__)):
    for j in range(len(snps)):
        if result_final__['position'][i] == snps['position'][j]:
            result_final__['allele_1'][i] = snps['allele1'][j]
            result_final__['allele_2'][i] = snps['allele2'][j]
result_final__

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,type,correlation,p_value_corr,gene,albicans_name(s),cerevisiae_name(s),allele_1,allele_2
0,1576666,SPAC23H4.17c,srb10,gene,I,1576712,1578028,+,None,0.05,snp,-0.118679,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T
1,1576666,SPAC23H4.17c,SPAC23H4.17c,mRNA,I,1576712,1578028,+,None,0.05,snp,-0.118679,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T
2,1576666,SPAC23H4.17c,SPAC23H4.17c.1,five_prime_UTR,I,1576712,1576853,+,five_prime_UTR,0.05,snp,-0.118679,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T
3,1576666,SPAC23H4.17c,SPAC23H4.17c.1,CDS,I,1576854,1577836,+,exon,0.05,snp,-0.118679,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T
4,1576666,SPAC23H4.17c,SPAC23H4.17c.1,intron,I,1577837,1577901,+,intron,0.05,snp,-0.118679,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2317428,SPBC12D12.07c,SPBC12D12.07c.1,CDS,II,2317984,2318251,-,exon,0.05,indel,0.125263,0.348806,trx2,NaN,NaN,AT,A
2274,2317428,SPBC12D12.07c,SPBC12D12.07c.1,three_prime_UTR,II,2317361,2317983,-,three_prime_UTR,0.05,indel,0.125263,0.348806,trx2,NaN,NaN,AT,A
2275,2317428,SPNCRNA.5597,None,gene,II,2317991,2318277,+,None,0.05,indel,NaN,NaN,NaN,NaN,NaN,AT,A
2276,2317428,SPNCRNA.5597,SPNCRNA.5597,ncRNA,II,2317991,2318277,+,None,0.05,indel,NaN,NaN,NaN,NaN,NaN,AT,A


In [8]:
import sys, re

f=open('/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Schizosaccharomyces_pombe_all_chromosomes.fa','r')
lines=f.readlines()

hre=re.compile('>(\S+)')
lre=re.compile('^(\S+)$')

regions={}

for line in lines:
        outh = hre.search(line)
        if outh:
                id=outh.group(1)
        else:
                outl=lre.search(line)
                if(id in regions.keys()):
                        regions[id] += outl.group(1)
                else:
                        regions[id]  =outl.group(1)

In [9]:
#result_final___ only keep rows where snp is between start and end
result_final___ = result_final__.copy()
result_final___ = result_final___[(result_final___['position'] >= result_final___['start']) & (result_final___['position'] <= result_final___['end'])]

gene_exons = df.copy()
#only keep rows where object is CDS
gene_exons = gene_exons[gene_exons['object'] == 'CDS']
#only keep rows where ID is in result_final__['ID']
gene_exons_final = pd.DataFrame()
for i in np.unique(result_final___['ID']):
      #keep ones where ID is i or i.1
      df_subset = gene_exons[(gene_exons['ID'] == i) | (gene_exons['ID'] == i + ".1")]
      gene_exons_final = gene_exons_final.append(df_subset)
#reset index
gene_exons_final.reset_index(inplace=True)
#make a column exon_number
gene_exons_final['exon_number'] = np.nan
#for those where ID is multiple times, number the exons by increasing start if strand is +, or decreasing start if strand is -
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    if df_subset['strand'].values[0]== '+':
        df_subset = df_subset.sort_values(by=['start'], ascending=True)
    if df_subset['strand'].values[0] == '-':
        df_subset = df_subset.sort_values(by=['end'], ascending=False)
    df_subset['exon_number'] = np.arange(1, len(df_subset) + 1)
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset


#REMOVE entries wehre ID contains SPNCRNA
gene_exons_final = gene_exons_final[~gene_exons_final['ID'].str.contains('SPNCRNA')]





#make a column called real_start
gene_exons_final['old_start'] = gene_exons_final['start']
gene_exons_final['old_end'] = gene_exons_final['end']
#for rows where strand is + look at the load the bases +- 5 around the start from regions
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    df_subset = df_subset[df_subset['exon_number'] == 1]
    if df_subset['strand'].values[0]== '+':
        seq = regions[df_subset['chromosome'].values[0]][int(df_subset['start'].values[0]-5):int(df_subset['start'].values[0]+5)]
        #where does ATG occur in seq
        real_start = seq.find('ATG')-4
        real_start = df_subset['start'] + real_start
        #for the row in gene_exons_final where ID is i and exon_number is 1, change the start to real_start
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == 1), 'start'] = real_start


        



    else:
        seq_rev = regions[df_subset['chromosome'].values[0]][int(df_subset['end'].values[0]-5):int(df_subset['end'].values[0]+5)]
        real_end = seq_rev.find('CAT')+2-4
        real_end = df_subset['end'] + real_end
        #for the row in gene_exons_final where ID is i and exon_number is 1, change the end to real_end
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == 1), 'end'] = real_end


    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    highest_exon = df_subset['exon_number'].max()
    df_subset = df_subset[df_subset['exon_number'] == highest_exon]
    if df_subset['strand'].values[0]== '+':

        seq2 = regions[df_subset['chromosome'].values[0]][int(df_subset['end'].values[0]-5):int(df_subset['end'].values[0]+5)]
        real_end = seq2.find('TAA')+2-4
        real_end = df_subset['end'] + real_end
        #for the row in gene_exons_final where ID is i and exon_number is highest_exon, change the end to real_end
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == highest_exon), 'end'] = real_end
    else:
        seq2_rev = regions[df_subset['chromosome'].values[0]][int(df_subset['start'].values[0]-5):int(df_subset['start'].values[0]+5)]
        real_start = seq2_rev.find('TCA')+2-4
        real_start = df_subset['start'] + real_start
        #for the row in gene_exons_final where ID is i and exon_number is highest_exon, change the start to real_start
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == highest_exon), 'start'] = real_start


    
    



#for each row calculate the length of the exon
gene_exons_final['exon_length'] = abs(gene_exons_final['end'] - gene_exons_final['start'])+1

#remove rows where ID is NaN
gene_exons_final = gene_exons_final[~gene_exons_final['ID'].isna()]
#for each row calculate the length of the CDS until that exon starts
gene_exons_final['CDS_length_so_far'] = np.nan
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    #order by exon_number
    df_subset = df_subset.sort_values(by=['exon_number'])
    
    df_subset['CDS_length_so_far'] = np.append(np.array(0),df_subset['exon_length'].cumsum())[:-1]
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset
#add total cds length
gene_exons_final['total_CDS_length'] = np.nan
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    df_subset['total_CDS_length'] = sum(df_subset['exon_length'])
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset

gene_exons_final


,index,chromosome,object,start,end,strand,Parent,ID,detail,exon_number,old_start,old_end,exon_length,CDS_length_so_far,total_CDS_length
0,7629,I,CDS,1578314,1578445,+,SPAC23H4.16c.1,SPAC23H4.16c.1,exon,1.0,1578314,1578445,132,0.0,984.0
1,7631,I,CDS,1578492,1578656,+,SPAC23H4.16c.1,SPAC23H4.16c.1,exon,2.0,1578492,1578656,165,132.0,984.0
2,7633,I,CDS,1578793,1578984,+,SPAC23H4.16c.1,SPAC23H4.16c.1,exon,3.0,1578793,1578984,192,297.0,984.0
3,7635,I,CDS,1579043,1579537,+,SPAC23H4.16c.1,SPAC23H4.16c.1,exon,4.0,1579043,1579540,495,489.0,984.0
4,7640,I,CDS,1576854,1577836,+,SPAC23H4.17c.1,SPAC23H4.17c.1,exon,1.0,1576854,1577836,983,0.0,1107.0
5,7642,I,CDS,1577902,1578025,+,SPAC23H4.17c.1,SPAC23H4.17c.1,exon,2.0,1577902,1578028,124,983.0,1107.0
6,7646,I,CDS,1575410,1575484,+,SPAC23H4.18c.1,SPAC23H4.18c.1,exon,1.0,1575410,1575484,75,0.0,324.0
7,7648,I,CDS,1575673,1575822,+,SPAC23H4.18c.1,SPAC23H4.18c.1,exon,2.0,1575673,1575822,150,75.0,324.0
8,7650,I,CDS,1575886,1575976,+,SPAC23H4.18c.1,SPAC23H4.18c.1,exon,3.0,1575886,1575976,91,225.0,324.0
9,7652,I,CDS,1576021,1576028,+,SPAC23H4.18c.1,SPAC23H4.18c.1,exon,4.0,1576021,1576028,8,316.0,324.0


In [10]:
#iterate through positions in result_final__ and look up the chromosome and position in regions, and find the corresponding letter +/- 5 bp in regions. Add it to a new column
result_final__['reference_seq']=np.nan
for i in range(len(result_final__)):
    for j in range(len(regions)):
        if result_final__['chromosome'][i] == list(regions.keys())[j]:
            #add | | characters left and right of tletter at position-1
            
            result_final__['reference_seq'][i] = regions[list(regions.keys())[j]][result_final__['position'][i]-4:result_final__['position'][i]-1] + ' |' + regions[list(regions.keys())[j]][result_final__['position'][i]-1] + '| ' + regions[list(regions.keys())[j]][result_final__['position'][i]:result_final__['position'][i]+3] 

#if strand is -, reverse reference_seq
for i in range(len(result_final__)):
    if result_final__['strand'][i] == '-':
        result_final__['reference_seq'][i] = result_final__['reference_seq'][i][::-1]
        #also flip the alleles
        result_final__['allele_1'][i] = result_final__['allele_1'][i][::-1]
        result_final__['allele_2'][i] = result_final__['allele_2'][i][::-1]
#also create reference_seq_ without the | | characters
result_final__['reference_seq_'] = result_final__['reference_seq'].str.replace('|', '')
result_final__['reference_seq_'] = result_final__['reference_seq_'].str.replace(' ', '')




In [12]:
result_final__

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,...,p_value_corr,gene,albicans_name(s),cerevisiae_name(s),allele_1,allele_2,reference_seq,reference_seq_,wt_gene,mutant allele
0,1576666,SPAC23H4.17c,srb10,gene,I,1576712,1578028,+,None,0.05,...,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T,TTC |G| CCT,TTCGCCT,NaN,NaN
1,1576666,SPAC23H4.17c,SPAC23H4.17c,mRNA,I,1576712,1578028,+,None,0.05,...,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T,TTC |G| CCT,TTCGCCT,NaN,NaN
2,1576666,SPAC23H4.17c,SPAC23H4.17c.1,five_prime_UTR,I,1576712,1576853,+,five_prime_UTR,0.05,...,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T,TTC |G| CCT,TTCGCCT,NaN,NaN
3,1576666,SPAC23H4.17c,SPAC23H4.17c.1,CDS,I,1576854,1577836,+,exon,0.05,...,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T,TTC |G| CCT,TTCGCCT,NaN,NaN
4,1576666,SPAC23H4.17c,SPAC23H4.17c.1,intron,I,1577837,1577901,+,intron,0.05,...,0.374919,srb10,['C2_04260W_A'],['YPL042C'],G,T,TTC |G| CCT,TTCGCCT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2317428,SPBC12D12.07c,SPBC12D12.07c.1,CDS,II,2317984,2318251,-,exon,0.05,...,0.348806,trx2,NaN,NaN,TA,A,TTT |A| CAT,TTTACAT,NaN,NaN
2274,2317428,SPBC12D12.07c,SPBC12D12.07c.1,three_prime_UTR,II,2317361,2317983,-,three_prime_UTR,0.05,...,0.348806,trx2,NaN,NaN,TA,A,TTT |A| CAT,TTTACAT,NaN,NaN
2275,2317428,SPNCRNA.5597,None,gene,II,2317991,2318277,+,None,0.05,...,NaN,NaN,NaN,NaN,AT,A,TAC |A| TTT,TACATTT,NaN,NaN
2276,2317428,SPNCRNA.5597,SPNCRNA.5597,ncRNA,II,2317991,2318277,+,None,0.05,...,NaN,NaN,NaN,NaN,AT,A,TAC |A| TTT,TACATTT,NaN,NaN


In [14]:
#create column, whole gene as codons
result_final__['wt_gene'] = np.nan
#iterate through result_final__ positions. If object is CDS, then look up the sequence from start to end in regions.
#add the sequence to the df
for i in range(len(result_final__)):
    
    if result_final__['object'][i] == 'CDS':
    
        start = result_final__['start'][i]
        end = result_final__['end'][i]
        chrom = result_final__['chromosome'][i]
        pos = result_final__['position'][i]
        gene = regions[chrom][start-1:end]
        #gene_exons_final where start is the same as start in result_final__
        exon = gene_exons_final[(gene_exons_final['start'] < start+20) & (gene_exons_final['end'] > end-20)] 
        #if exon is empty, then print gene name
        if exon.empty:
            print(result_final__['ID'][i])
            continue
        exon_len_so_far=0
        exon_len_so_far = exon['CDS_length_so_far'].values[0]
        exon_len_so_far=exon_len_so_far%3
        if pos > start and pos < end:
        
            if result_final__['strand'][i] == '-':
                
                gene = gene[::-1]
            
                gene = ''.join(gene)
            gene = 'A'*int(exon_len_so_far) + gene
            #write the gene as triplets of letters like AAA, ATC, etc
            #if divisible by 3
            if len(gene)%3 != 0:
                gene = gene[:-(len(gene)%3)]
            gene = [gene[i:i+3] for i in range(0, len(gene), 3)]
                #only triplets near the position
            if result_final__['strand'][i] == '+':
                gene = gene[(pos-(start-int(exon_len_so_far)))//3-2:(pos-end)//3+25]
            else: 
                gene = gene[(end-pos)//3-2:(start-int(exon_len_so_far)-pos)//3+25]
            
                
                


            result_final__['wt_gene'][i] = gene

result_final__['mutant allele'] = np.nan
for i in range(len(result_final__)):
    if result_final__['object'][i] == 'CDS':
    
        start = result_final__['start'][i]
        end = result_final__['end'][i]
        chrom = result_final__['chromosome'][i]
        pos = result_final__['position'][i]
        gene = regions[chrom][start-1:end]
         #gene_exons_final where start is the same as start in result_final__
        exon = gene_exons_final[(gene_exons_final['start'] < start+20) & (gene_exons_final['end'] > end-20)] 
        #if exon is empty, then print gene name
        if exon.empty:
            print(result_final__['ID'][i])
            continue
        exon_len_so_far=0
        exon_len_so_far = exon['CDS_length_so_far'].values[0]
        exon_len_so_far=exon_len_so_far%3
        if pos > start and pos < end:
        
            if result_final__['strand'][i] == '+':
                #decide which allele to use
                #check 
                #check how long the y0036 allele is 
                x968_len = len(result_final__['allele_1'][i])
                y0036_len = len(result_final__['allele_2'][i])
                
                mutation = result_final__['allele_2'][i]
               

                gene = gene[:pos-start] + mutation + gene[pos-start+x968_len:]
            elif result_final__['strand'][i] == '-':
                x968_len = len(result_final__['allele_1'][i])
                y0036_len = len(result_final__['allele_2'][i])
                gene = gene[::-1]
                gene = ''.join(gene)
                mutation = result_final__['allele_2'][i]
                
                gene = gene[:end-pos] + mutation + gene[end-pos+x968_len:]
                
            gene = 'A'*int(exon_len_so_far) + gene
            if len(gene)%3 != 0:
                gene = gene[:-(len(gene)%3)]
            gene = [gene[k:k+3] for k in range(0, len(gene), 3)]
                #only triplets near the position
            if result_final__['strand'][i] == '+':
                gene = gene[(pos-(start-int(exon_len_so_far)))//3-2:(pos-end)//3+25]
            else:
                gene = gene[(end-pos)//3-2:(start-int(exon_len_so_far)-pos)//3+25]
            result_final__["mutant allele"][i] = gene

SPNCRNA.2696
SPNCRNA.2697
SPNCRNA.2698
SPNCRNA.2696
SPNCRNA.2697
SPNCRNA.2698
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPAC16A10.08c
SPAC16A10.08c
SPNCRNA.885
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPNCRNA.886
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNCRNA.3393
SPNCRNA.3394
SPAC688.07c
SPNC

In [15]:
#show all rows of results_final__
pd.set_option('display.max_rows', None)
#order and by p_value_corr
result_final__ = result_final__.sort_values(by=['p_value_corr'],ascending=True)
result_final__

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,...,p_value_corr,gene,albicans_name(s),cerevisiae_name(s),allele_1,allele_2,reference_seq,reference_seq_,wt_gene,mutant allele
990,2316009,SPBC12D12.09,rev7,gene,II,2313096,2314605,+,None,0.05,...,0.019359,rev7,NaN,NaN,A,G,TGT |A| CCG,TGTACCG,NaN,NaN
991,2316009,SPBC12D12.09,SPBC12D12.09,mRNA,II,2313096,2314605,+,None,0.05,...,0.019359,rev7,NaN,NaN,A,G,TGT |A| CCG,TGTACCG,NaN,NaN
992,2316009,SPBC12D12.09,SPBC12D12.09.1,three_prime_UTR,II,2314477,2314605,+,three_prime_UTR,0.05,...,0.019359,rev7,NaN,NaN,A,G,TGT |A| CCG,TGTACCG,NaN,NaN
837,3126179,SPAC688.08,SPAC688.08,mRNA,I,3123163,3126653,+,None,0.05,...,0.086576,srb8,NaN,NaN,A,T,ATC |A| GCT,ATCAGCT,NaN,NaN
669,3124051,SPAC688.08,srb8,gene,I,3123163,3126653,+,None,0.05,...,0.086576,srb8,NaN,NaN,T,C,AAA |T| GTG,AAATGTG,NaN,NaN
659,3124046,SPAC688.08,SPAC688.08.1,CDS,I,3125118,3126310,+,exon,0.05,...,0.086576,srb8,NaN,NaN,G,A,GTC |G| TAA,GTCGTAA,NaN,NaN
658,3124046,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,...,0.086576,srb8,NaN,NaN,G,A,GTC |G| TAA,GTCGTAA,"[ATT, TGT, CGT, AAA, TGT, GTT, ATC, ACA, ATT, ...","[ATT, TGT, CAT, AAA, TGT, GTT, ATC, ACA, ATT, ..."
657,3124046,SPAC688.08,SPAC688.08,mRNA,I,3123163,3126653,+,None,0.05,...,0.086576,srb8,NaN,NaN,G,A,GTC |G| TAA,GTCGTAA,NaN,NaN
656,3124046,SPAC688.08,srb8,gene,I,3123163,3126653,+,None,0.05,...,0.086576,srb8,NaN,NaN,G,A,GTC |G| TAA,GTCGTAA,NaN,NaN
646,3123901,SPAC688.08,SPAC688.08.1,CDS,I,3125118,3126310,+,exon,0.05,...,0.086576,srb8,NaN,NaN,C,A,TTG |C| TGG,TTGCTGG,NaN,NaN


In [16]:
#CreATE A DF WHERE WHOLE GENE IS NOT NAN
stop_codon_df = result_final__[result_final__['wt_gene'].notna()]
stop_codon_df["stop_codon_predicted"] = np.nan
#iterate through mutant allele, and see whether it contains
#reset index
stop_codon_df = stop_codon_df.reset_index(drop=True)

for i in range(len(stop_codon_df)):
    for j in range(len(stop_codon_df['mutant allele'][i])):
        if stop_codon_df["strand"][i] == '+':
            if stop_codon_df['mutant allele'][i][j] == 'TAA' or stop_codon_df['mutant allele'][i][j] == 'TAG' or stop_codon_df['mutant allele'][i][j] == 'TGA':
                stop_codon_df['stop_codon_predicted'][i] = 'yes'
        else:
            if stop_codon_df['mutant allele'][i][j] == 'ATT' or stop_codon_df['mutant allele'][i][j] == 'ATC' or stop_codon_df['mutant allele'][i][j] == 'ACT':
                stop_codon_df['stop_codon_predicted'][i] = 'yes'
#show which entries have a stop codon predicted
stop_codon_df[stop_codon_df['stop_codon_predicted'] == 'yes']

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,...,gene,albicans_name(s),cerevisiae_name(s),allele_1,allele_2,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted
54,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,srb11,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes
55,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,srb11,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes
62,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,srb11,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes
64,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,srb11,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes
66,3097513,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,...,NaN,NaN,NaN,C,T,TTC |C| AAC,TTCCAAC,"[GTT, ATT, CCA, ACA, TCG, TCT, ATT, TCC, TCC, ...","[GTT, ATT, CTA, ACA, TCG, TCT, ATT, TCC, TCC, ...",yes
67,3097633,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,...,NaN,NaN,NaN,C,T,TAA |C| GGG,TAACGGG,"[TAA, TTA, ACG, GGA, GAA, GAA, CTA, TGA, GAA, ...","[TAA, TTA, ATG, GGA, GAA, GAA, CTA, TGA, GAA, ...",yes
68,3097633,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,...,NaN,NaN,NaN,C,T,TAA |C| GGG,TAACGGG,"[TAA, TTA, ACG, GGA, GAA, GAA, CTA, TGA, GAA, ...","[TAA, TTA, ATG, GGA, GAA, GAA, CTA, TGA, GAA, ...",yes
72,3097927,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,...,NaN,NaN,NaN,A,C,TTG |A| AGT,TTGAAGT,"[CAC, TTT, GAA, GTA, GTC, GTA, AGA, GAG, AAT, ...","[CAC, TTT, GCA, GTA, GTC, GTA, AGA, GAG, AAT, ...",yes
73,3097934,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,...,NaN,NaN,NaN,C,A,AGT |C| GTA,AGTCGTA,"[GAA, GTA, GTC, GTA, AGA, GAG, AAT, CCA, TAG, ...","[GAA, GTA, GTA, GTA, AGA, GAG, AAT, CCA, TAG, ...",yes
74,3098103,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,...,NaN,NaN,NaN,G,A,CTT |G| TAG,CTTGTAG,"[TGC, CTT, GTA, GGC, TTT, AAG, TAA, AAT, TCT, ...","[TGC, CTT, ATA, GGC, TTT, AAG, TAA, AAT, TCT, ...",yes


In [18]:
#in stop_codon_df create a column "possible frameshift" and fill it with nan. Then loop through X96_allele and Y0036_allele and check if the length is different. If yes and the difference is not divisible by 3, fill the column with "yes". Then show the entries where the column is "yes"
frameshift_df = stop_codon_df.copy()
frameshift_df['possible frameshift'] = np.nan
for i in range(len(frameshift_df)):
    if len(frameshift_df['allele_1'][i]) != len(frameshift_df['allele_2'][i]):
        if (len(frameshift_df['allele_1'][i]) - len(frameshift_df['allele_2'][i]))%3 != 0:
            frameshift_df['possible frameshift'][i] = 'yes'
frameshift_df[frameshift_df['possible frameshift'] == 'yes']

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,...,albicans_name(s),cerevisiae_name(s),allele_1,allele_2,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted,possible frameshift
54,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes,yes
55,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes,yes
56,2316665,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316612,2316691,+,exon,0.05,...,['C3_05740C_A'],['YNL025C'],CTA,C,TAT |C| TAA,TATCTAA,[],[],NaN,yes
57,2316665,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316612,2316691,+,exon,0.05,...,['C3_05740C_A'],['YNL025C'],CTA,C,TAT |C| TAA,TATCTAA,[],[],NaN,yes
62,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes,yes
64,2316851,SPBC12D12.06,SPBC12D12.06.1,CDS,II,2316770,2317136,+,exon,0.05,...,['C3_05740C_A'],['YNL025C'],TA,T,TGC |T| AAA,TGCTAAA,"[TAC, ATG, CTA, AAA, AAA, AAT, GAG, GAA, AAG, ...","[TAC, ATG, CTA, AAA, AAA, ATG, AGG, AAA, AGG, ...",yes,yes


In [19]:
#import a codon table as a dictionary
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}
#iterate through frameshift_df and trnslate the lists of triplets found in wt_gene and mutant allele
frameshift_df['wt_gene_translated'] = np.nan
frameshift_df['mutant allele_translated'] = np.nan
for i in range(len(frameshift_df)):
    #if strand is -, then change a to t and so on
    if frameshift_df['strand'][i] == '-':
        temp_wt_allele = frameshift_df['wt_gene'][i]
        temp_mut_allele = frameshift_df['mutant allele'][i]
        for j in range(len(temp_wt_allele)):
            temp_wt_allele[j] = temp_wt_allele[j].replace('A','t')
            temp_wt_allele[j] = temp_wt_allele[j].replace('T','a')
            temp_wt_allele[j] = temp_wt_allele[j].replace('C','g')
            temp_wt_allele[j] = temp_wt_allele[j].replace('G','c')
            temp_wt_allele[j] = temp_wt_allele[j].upper()
        for j in range(len(temp_mut_allele)):
            temp_mut_allele[j] = temp_mut_allele[j].replace('A','t')
            temp_mut_allele[j] = temp_mut_allele[j].replace('T','a')
            temp_mut_allele[j] = temp_mut_allele[j].replace('C','g')
            temp_mut_allele[j] = temp_mut_allele[j].replace('G','c')
            temp_mut_allele[j] = temp_mut_allele[j].upper()
        frameshift_df['wt_gene_translated'][i] = ''.join([codon_table[codon] for codon in temp_wt_allele])
        frameshift_df['mutant allele_translated'][i] = ''.join([codon_table[codon] for codon in temp_mut_allele])
    else:
        frameshift_df['wt_gene_translated'][i] = ''.join([codon_table[codon] for codon in frameshift_df['wt_gene'][i]])
        frameshift_df['mutant allele_translated'][i] = ''.join([codon_table[codon] for codon in frameshift_df['mutant allele'][i]])

        
    

In [20]:
frameshift_df['non-synonymous mutation'] = "no"
for i in range(len(frameshift_df)):
      if frameshift_df["wt_gene_translated"][i] != frameshift_df["mutant allele_translated"][i]:
            frameshift_df['non-synonymous mutation'][i] = 'yes'

In [21]:
#show for entries where ID doesnt start wiht SPNCRNA
#show all columns
pd.set_option('display.max_columns', None)
#show me rows where the difference is not a D TO N mutation
#add 2 columns: aa wt to aa mut. iterate through frameshift_df and add the third value from each wt_gene_translated and mutant allele_translated to aa_wt and aa_mut
#reset index
#make column aa_wt and aa_mut
frameshift_df['aa_wt'] = np.nan
frameshift_df['aa_mut'] = np.nan
frameshift_df.reset_index(inplace = True,drop=True)
for i in range(len(frameshift_df)):
      if frameshift_df['wt_gene_translated'][i] == "":
            continue
      frameshift_df['aa_wt'][i] = frameshift_df['wt_gene_translated'][i][3]
      frameshift_df['aa_mut'][i] = frameshift_df['mutant allele_translated'][i][3]



In [22]:
frameshift_df

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,type,correlation,p_value_corr,gene,albicans_name(s),cerevisiae_name(s),allele_1,allele_2,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted,possible frameshift,wt_gene_translated,mutant allele_translated,non-synonymous mutation,aa_wt,aa_mut
0,3124046,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,G,A,GTC |G| TAA,GTCGTAA,"[ATT, TGT, CGT, AAA, TGT, GTT, ATC, ACA, ATT, ...","[ATT, TGT, CAT, AAA, TGT, GTT, ATC, ACA, ATT, ...",NaN,NaN,ICRKCVITILQSYQQLIVDQPANLLKF,ICHKCVITILQSYQQLIVDQPANLLKF,yes,K,K
1,3123901,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,C,A,TTG |C| TGG,TTGCTGG,"[CGA, TTG, CTG, GCT, GAA, GAG, TTG, GTT, TCG, ...","[CGA, TTG, ATG, GCT, GAA, GAG, TTG, GTT, TCG, ...",NaN,NaN,RLLAEELVSPKHFLLRIVSFSGDSSLK,RLMAEELVSPKHFLLRIVSFSGDSSLK,yes,A,A
2,3124051,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,T,C,AAA |T| GTG,AAATGTG,"[CGT, AAA, TGT, GTT, ATC, ACA, ATT, CTG, CAA, ...","[CGT, AAA, CGT, GTT, ATC, ACA, ATT, CTG, CAA, ...",NaN,NaN,RKCVITILQSYQQLIVDQPANLLKFSL,RKRVITILQSYQQLIVDQPANLLKFSL,yes,V,V
3,3123847,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,A,G,GAT |A| TTT,GATATTT,"[GCT, GAT, ATT, TAC, TCG, TTC, GCT, GAT, GAT, ...","[GCT, GAT, GTT, TAC, TCG, TTC, GCT, GAT, GAT, ...",NaN,NaN,ADIYSFADDYTYLLKLFGRLLAEELVS,ADVYSFADDYTYLLKLFGRLLAEELVS,yes,Y,Y
4,3123786,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,T,C,AAA |T| TGG,AAATTGG,"[GTT, AAA, AAT, TGG, ACT, GAT, GTT, GCA, GCG, ...","[GTT, AAA, AAC, TGG, ACT, GAT, GTT, GCA, GCG, ...",NaN,NaN,VKNWTDVAAGFLISFISSFLNADIYSFA,VKNWTDVAAGFLISFISSFLNADIYSFA,no,W,W
5,3123611,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,C,T,CTT |C| TCT,CTTCTCT,"[TTG, TCT, TCT, CTT, ATT, AAA, ACT, ATA, CCA, ...","[TTG, TCT, TTT, CTT, ATT, AAA, ACT, ATA, CCA, ...",NaN,NaN,LSSLIKTIPHGLWGEDILRMLVKFRVP,LSFLIKTIPHGLWGEDILRMLVKFRVP,yes,L,L
6,3123847,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,A,G,GAT |A| TTT,GATATTT,"[GCT, GAT, ATT, TAC, TCG, TTC, GCT, GAT, GAT, ...","[GCT, GAT, GTT, TAC, TCG, TTC, GCT, GAT, GAT, ...",NaN,NaN,ADIYSFADDYTYLLKLFGRLLAEELVS,ADVYSFADDYTYLLKLFGRLLAEELVS,yes,Y,Y
7,3124473,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,T,A,AGG |T| GAT,AGGTGAT,"[TTT, TCA, GGT, GAT, CGA, AAT, TTC, ATC, GTA, ...","[TTT, TCA, GGA, GAT, CGA, AAT, TTC, ATC, GTA, ...",NaN,NaN,FSGDRNFIVSSIFDNANFDRNLIVDLTL,FSGDRNFIVSSIFDNANFDRNLIVDLTL,no,D,D
8,3124473,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,T,A,AGG |T| GAT,AGGTGAT,"[TTT, TCA, GGT, GAT, CGA, AAT, TTC, ATC, GTA, ...","[TTT, TCA, GGA, GAT, CGA, AAT, TTC, ATC, GTA, ...",NaN,NaN,FSGDRNFIVSSIFDNANFDRNLIVDLTL,FSGDRNFIVSSIFDNANFDRNLIVDLTL,no,D,D
9,3124328,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.227019,0.086576,srb8,NaN,NaN,T,C,GTA |T| TCG,GTATTCG,"[AGT, AGT, ATT, CGC, TTC, TTG, CAA, ATT, ATT, ...","[AGT, AGT, ACT, CGC, TTC, TTG, CAA, ATT, ATT, ...",NaN,NaN,SSIRFLQIISCLSFPVGWRTLAKDLFK,SSTRFLQIISCLSFPVGWRTLAKDLFK,yes,R,R


In [ ]:
final_final_df = frameshift_df[frameshift_df['non-synonymous mutation'] == 'yes']
#remove rows where aa_wt is D and aa_mut is N or vice versa
def remove_aa_pairs(df,aa1,aa2):
      for i in df.index:
            if df['aa_wt'][i] == aa1 and df['aa_mut'][i] == aa2:
                  df.drop(i,inplace=True)
            elif df['aa_wt'][i] == aa2 and df['aa_mut'][i] == aa1:
                  df.drop(i,inplace=True)
      df.reset_index(inplace = True,drop=True)

      return df

final_final_df = remove_aa_pairs(final_final_df,'I','V')
final_final_df = remove_aa_pairs(final_final_df,'I','L')
final_final_df = remove_aa_pairs(final_final_df,'V','L')
final_final_df = remove_aa_pairs(final_final_df,'D','E')
final_final_df



,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,type,correlation,p_value_corr,gene,albicans_name(s),cerevisiae_name(s),allele_1,allele_2,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted,possible frameshift,wt_gene_translated,mutant allele_translated,non-synonymous mutation,aa_wt,aa_mut
0,3123179,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.198403,0.135441,srb8,['C4_05090C_A'],['YCR081W'],C,T,ATT |C| AGC,ATTCAGC,"[AAT, CGA, GAT, TCA, GCA, AAC]","[AAT, CGA, GAT, TTA, GCA, AAC]",NaN,NaN,NRDSAN,NRDLAN,yes,S,L
1,3123901,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.198403,0.135441,srb8,['C4_05090C_A'],['YCR081W'],C,A,TTG |C| TGG,TTGCTGG,"[GGT, CGA, TTG, CTG, GCT, GAA]","[GGT, CGA, TTG, ATG, GCT, GAA]",NaN,NaN,GRLLAE,GRLMAE,yes,L,M
2,3124051,SPAC688.08,SPAC688.08.1,CDS,I,3123163,3125118,+,exon,0.05,snp,-0.198403,0.135441,srb8,['C4_05090C_A'],['YCR081W'],T,C,AAA |T| GTG,AAATGTG,"[TGT, CGT, AAA, TGT, GTT, ATC]","[TGT, CGT, AAA, CGT, GTT, ATC]",NaN,NaN,CRKCVI,CRKRVI,yes,C,R
3,3095581,SPAC589.02c,SPAC589.02c.1,CDS,I,3095075,3098590,-,exon,0.05,snp,-0.100217,0.454157,med13,['C2_01470W_A'],['YDR443C'],C,T,CTG |C| GTC,CTGCGTC,"[ATG, TCT, GAC, GCA, GAG, TAT]","[ATG, TCT, GAC, ACA, GAG, TAT]",NaN,NaN,MSDAEY,MSDTEY,yes,A,T
4,3097633,SPNCRNA.3387,SPNCRNA.3387.1,CDS,I,3096807,3098568,+,exon,0.05,snp,NaN,NaN,NaN,NaN,NaN,C,T,TAA |C| GGG,TAACGGG,"[GAA, TAA, TTA, ACG, GGA, GAA]","[GAA, TAA, TTA, ATG, GGA, GAA]",yes,NaN,E*LTGE,E*LMGE,yes,T,M


In [ ]:
#iterate through snps unnamed: 0
plots=[]
for i in range(len(snps_final)):
    #if type is snp
    if snps_final["type"][i] == "snp":
        loc=snps_final["position"][i]
        strains = snpsdata["samples"][snpsdata['calldata/GT'][np.where(snpsdata['variants/POS'] == float(loc))[0][0],:,0] == 1]
    else:
        loc=snps["position"][i]
        strains =  indelsdata["samples"][indelsdata['calldata/GT'][np.where( indelsdata['variants/POS']== float(loc))[0][0],:,0] == 1]

    #find strains ratio value of strains in wash_data
    mutated_vals = wash_data[wash_data["strain"].isin(strains)]["ratio"]
    wt_vals = wash_data[~wash_data["strain"].isin(strains)]["ratio"]
    #find p value
    p = st.ttest_ind(mutated_vals,wt_vals)[1]
    #make plot if p_value < 0.05
    if p < 0.01:
        #create df with values and mut or wt
        df = pd.DataFrame({"Ratio":np.concatenate((wt_vals,mutated_vals)),"Mutated":np.concatenate((np.repeat("Ref. allele",len(wt_vals)),np.repeat("Mutant",len(mutated_vals))))})
        genes = np.unique(result_final__.loc[result_final__["position"] == int(loc)]["gene"].dropna().values)
        #put genes in title but make it look not like a list, but keep commas
        plots.append(iqplot.stripbox(df,"Ratio","Mutated",title=str(snps["position"][i]) + "  genes: " + str(genes).replace("[","").replace("]","").replace("'","").replace(" ",", "),width=600,height=300, q_axis="y"))
#allow bokeh output in notebook
bokeh.io.output_notebook()
bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=5))

NameError: name 'snps_final' is not defined

In [ ]:
#give me a gridplot wiht plot 1,2,5,9
plots_retain = [plots[0],plots[1],plots[4],plots[7], plots[17], plots[10]]
bokeh.io.show(bokeh.layouts.gridplot(plots_retain,ncols=3))

IndexError: list index out of range

In [ ]:
snpsdata["samples"][snpsdata['calldata/GT'][np.where(snpsdata['variants/POS'] == 2227937)[0][0],:,0] == 1]

array(['JB1205', 'JB1206', 'JB1207', 'JB872', 'JB914', 'JB934'],
      dtype=object)

df_temp